In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

#Scrape the pro-football reference for super bowl scores
res = requests.get('https://www.pro-football-reference.com/super-bowl/')
soup = BeautifulSoup(res.content,'lxml')

links=[]

#Go through each superbowl page to scrape the box score.
for a in soup.find_all('a',href=True):
    if a.text:
        if 'boxscore' in a['href'] and hasNumbers(a['href']):
            links.append('https://www.pro-football-reference.com/'+a['href'])

#First link is not needed
links.pop(0)


#Initialize the quarter scores
q1 = []
q2 = []
q3 = []
qf = []


#for each link, determine the end of the quarter score. Note that the box score will not provide this, as you have to add the previous quarter score. 
for j in links:
    res2 = requests.get(j)
    soup = BeautifulSoup(res2.content,'lxml')
    table = pd.read_html(str(soup.find_all('table')[0]))[0]
    table = pd.DataFrame(table).loc[:,['1','2','3','Final']]
    
    
    
    table = table.apply(lambda x: x%10,axis = 1)
    
    total_score = [0,0]
    
    total_score = table.loc[:,'1'] + total_score
    
    q1.append(list(sorted(total_score%10)))
    total_score = table.loc[:,'2'] + total_score
    
    q2.append(list(sorted(total_score%10)))
    
    total_score = table.loc[:,'3'] + total_score
    q3.append(list(sorted(total_score%10)))
    
    total_score = table.loc[:,'Final']
    
    qf.append(list(sorted(total_score%10)))
    

#Create a data frame of the results.
finaldf2 = pd.DataFrame({'q1':q1,'q2':q2,'q3':q3,'qf':qf})

newf = finaldf2.astype('str')

newf.groupby(['q1','q2','q3','qf']).size().reset_index()

#newf.to_csv('C:/Users/Sam.Lederer/Desktop/testfb.csv')

f = pd.DataFrame({'n':q1+q2+q3+qf})
f = f.astype('str')

#calculate the percent break out
f.groupby('n').size()/(52*4)*100

n
[0, 0]    8.173077
[0, 1]    1.923077
[0, 2]    1.923077
[0, 3]    8.653846
[0, 4]    5.769231
[0, 5]    1.923077
[0, 6]    2.884615
[0, 7]    9.615385
[0, 8]    1.923077
[0, 9]    3.365385
[1, 3]    1.442308
[1, 4]    1.923077
[1, 5]    1.923077
[1, 6]    0.961538
[1, 7]    2.884615
[1, 8]    0.480769
[1, 9]    0.480769
[2, 2]    0.480769
[2, 4]    0.480769
[2, 7]    1.442308
[2, 8]    0.480769
[2, 9]    0.480769
[3, 3]    3.846154
[3, 4]    2.884615
[3, 6]    3.365385
[3, 7]    7.211538
[3, 8]    0.961538
[3, 9]    0.480769
[4, 4]    1.442308
[4, 6]    0.961538
[4, 7]    5.288462
[4, 8]    0.961538
[4, 9]    0.961538
[5, 7]    0.480769
[5, 9]    0.480769
[6, 6]    0.961538
[6, 7]    3.365385
[6, 8]    1.923077
[6, 9]    0.961538
[7, 7]    3.365385
[7, 9]    1.442308
[8, 9]    0.961538
dtype: float64